
__rel / nonRel 이란?__

rel / nonRel  * TF * IDF 

후드티 스웨터 드레스 
바지 신발 주방용품

* 분석 대상인 카테고리의 문서들과, 분석 대상이 아닌 카테고리의 문서들을 모두 가지고 있을 경우, 특정단어가 분석 대상인 카테고리에 해당하는 단어인지를 수치로 표현 하기 위한 방법.
* (근본없는 방법입니다.. 한번 시도해보고 별로면 팽)

* __등장배경__
    * IDF가 stopWords들을 잘 걸러주지만, 유의미한 단어들(옷의 특징이 되는 단어들이지만 많은 옷에 등장해서 idf값이 낮아진)또한 걸러낸다는 문제점을 보안하기 위함.
    
* __단점__
    * 쓸데없는 데이터까지 크롤링해야함.. 그만큼 연산도 오래걸림..bow 길이 늘어남.. 

__현재 상황__

* 일단은 필요없어 보이는 카테고리(치마,바지)의 데이터도 저장해둠.
* 분석대상은 스웨터, 드레스(+ 후디)
* 즉, 어떤 단어가 스웨터, 드레스(+후디)를 나타내는 단어인지 알아야 한다.
-----
1.  rel Category : 스웨터, 드레스
2.  nonRel Category : 치마, 바지
------
__방법__

1. 전체 카테고리(토픽)을 두 부류로 나눔 => (이 경우엔)옷과관련된 카테고리, 아닌 카테고리

2. 모든 단어에 rel / nonRel 값을 부여
    1. rel : 관련 카테고리에 등장한 횟수 
    2. nonRel : 관련x 카테고리에서 등장한 횟수
3. rel / nonRel : 
    * 이 값이 클수록? => 우리가 원하는 주제와 관련된 단어이다
    
   
# * 사용할 라이브러리 + 함수

In [ ]:
import json
from pathlib import Path  # Path클래스 임포트
import re  # 특수문자 제거
from functools import cmp_to_key
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words #stop word 제거
from nltk.stem.porter import PorterStemmer #어간 추출
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer #표제어 추출

stemmer = PorterStemmer() 
lemmatizer = WordNetLemmatizer()
en_stop = get_stop_words('en')

#내림차순 정렬을 위한 함수객체들
def cmp(l,r):
    if l>r:
        return 1
    elif l==r :
        return 0
    else:
        return -1
def key_func(l,r):
    if l[1]==r[1]:
        return cmp(l[0],r[0])
    return -cmp(l[1],r[1])

#dict()
#dictionary => 내림차순 정렬된 list of tuple으로 만들어서 리턴
def sortDict(bow):
    word_list=[] #리스트에 (단어, 횟수) 꼴의 튜플을 넣을 것

    for i,k in bow.items() :
        word_list.append((i,k))
    word_list.sort(key=lambda x:x[1], reverse=True) #횟수를 기준으로 내림차순 정렬
    word_list.sort(key=cmp_to_key(key_func))# 횟수는같을때, 단어를 기준으로 정렬
    return word_list
#list of tuple을 출력
def displayTupList(tupList, num, withVal=False):  # 디폴트는 내림차순, 값 없이..
    counter = 0
    for tup in tupList:
        if tup[1] == 0 or counter == num:
            break
        else:
            if withVal:
                print(tup[0], " : ", tup[1], "  ", end="")
            else:
                print(tup[0], "   ", end="")
            counter += 1
    print()
#어떤 특수기호를 제거할지
def delSymbol(feature):
    str = re.sub('[^A-Za-z0-9-]+', ' ', feature)  # 알파벳, 숫자, '-'  빼고 지우기
    str = re.sub('[0-9]+', 'ㅠ', str)  # 숫자를 포함한 단어 전체를 지우기 위함.
    while (str.find("--") != -1):  # ' - ' 두개이상은 지움
        str = str.replace("--", " ")
    return str.strip('-')  # 겉에남은 '-' 제거

#버릴단어
def isSemantic(word):
    if word == '' or word == '-'or word.find("ㅠ") != -1: #or word == '%' or word.isdigit()
        return False
    else:
        return True
    
def tokenize(feature): 
    tokens = []
    feature = delSymbol(feature)
    for token in feature.split():
        token = token.lower().strip('-')
        if isSemantic(token):
            tokens.append(token)
    tokens = [i for i in tokens if not i in en_stop] ###stop words 제거
    tokens = [lemmatizer.lemmatize(i) for i in tokens] ###표제어 추출
    return tokens

#경로에 있는 모든 id.json 파일의 features로 bag of word(딕셔너리)를 만들어 리턴
def makeBow(path) :
    for item in path.iterdir():
        if(item.is_dir()): #안보이는 디렉토리가 있다.. 주의
            continue
        itemJson = json.loads(item.read_text(), encoding='utf-8')
        features = itemJson["features"]
        allStr = ""
        for feature in features:
            allStr = allStr + " " + feature + " "
        tokens = tokenize(allStr)
        for token in tokens:
            if token in bow:
                bow[token] += 1
            else:
                bow[token] = 1
    return bow
#특정 문서(features)하나 만의 bag of word를 만들고, 단어개수 리턴.
def makeSmallBow(features) : 
    smallBow = {}
    counter = 0 #단어개수
    allStr = ""
    for feature in features:
        allStr = allStr + " " + feature + " "
    tokens = tokenize(allStr)
    for token in tokens:
        counter += 1
        if token in smallBow:
            smallBow[token] += 1
        else:
            smallBow[token] = 1
    return smallBow, counter

#문서 하나의 tf를 계산 (dictionary)                
def computeTF(bow, features):  
    smallBow, numOfWord = makeSmallBow(features)  # eachBow : 특정 상품의 word count dict
    # numOfWord : 특정 상품의 word 개수
    tf = {}  # 단어별 tf를 계산해서 넣음
    for word, count in smallBow.items():
        if (numOfWord == 0):  # feature 없는 상품..
            break
        # tf 어떻게 할까 ===============================================
        #tf[word] = count / float(numOfWord)
        tf[word] = 1 #단어가 있냐 없냐만 따짐
        #tf[word] = float(numOfWord) / count #역수 (문서 내 자주 등장한 단어는 중요하지 않다)
    return tf

#path 에 있는 모든 id.json 파일을 가지고, tf를 구하여 "ID" : tf   꼴의 딕셔너리 리턴
def computeAllTF(path):
    for item in path.iterdir():
        if(item.is_dir()): 
            continue
        itemJson = json.loads(item.read_text(), encoding='utf-8')
        features = itemJson["features"]
        ID = itemJson["id"]
        
        tf = computeTF(bow, features)
        tfDict[ID] = tf
    return tfDict

#모든 tf  가지고, idf 를 계산 (dictionary)
def computeIDF(tfs) : #수정----------------------------------------------------------------------
    import math
    idf = {}
    N = len(tfs)  # 모든 상품 개수
    for ID, tf in tfs.items(): 
        for word, val in tf.items():
            if val > 0:
                if word in idf:
                    idf[word] += 1  # 일단은 "word : word를 가진 상품개수" 꼴의 사전 완성..
                else:
                    idf[word] = 1
    for word, val in idf.items():
        idf[word] = math.log10(N / float(val))
    return idf

#문서 하나의 tf-idf 계산 (dictionary)
def computeTFIDF(tf, idfs): 
    tfidf = {}
    for word, val in tf.items():
        tfidf[word] = val * idfs[word]
    return tfidf
#(시도1) : tf / idf 
# 차라리 여기저기 자주 등장한 단어에게 높은 값을 주자..
def computeTFIDF2(tf, idfs):
    tfidf2 = {} 
    for word, val in tf.items():
        tfidf2[word] = val / idfs[word]
    return tfidf2

path1 = Path('/home/jovyan/work/크롤링/relFeatures')
path2 = Path('/home/jovyan/work/크롤링/nonRelFeatures')
paths = [path1, path2]


# 0. rel / nonRel 계산 

1. 아니.. non relative에서 한번도 안나온 단어면..? 중요한단어일수도있고 오타일수도 있고 -ㅡㅡ 으음..
    * 최댓값으로 설정? (586)
2. 값의 범위가 너무 넓음 ㅡㅡ : 0 ~ 586.. 
    * (rCount/rTotalWordCount) * (nCount/nTotalWordCount)

In [7]:
# 1. Path1, path2 를 통틀어 하나의 BOW 를 만든다. (사실 단어 내림차순 할 때 말고는 쓸모없음)
# 2. 동시에 relNonRel Dict 역시 만든다.  word : {rel: , nonRel: }
def bowAndRel(paths) :
    bow = {}
    relNonRelDict = {}  # word : {"R": , "N": }
    for path in paths:
        for item in path.iterdir():
            if(item.is_dir()): 
                continue
            itemJson = json.loads(item.read_text(), encoding='utf-8')
            features = itemJson["features"]
            category = itemJson["category"]
            RN = None
            if category == "sweater" or category == "hoodies&sweatshirts" or category == "dress":
                RN = "R"
            elif category == "skirt" or "pants" or "nonRel":
                RN = "N"
            else:
                print("에러!")
            featureStr = ""
            for feature in features:
                featureStr = featureStr + " " +  feature + " "
            tokens = tokenize(featureStr)
            for token in tokens:
                if token in bow:
                    bow[token] += 1
                else:
                    bow[token] = 1
                if token not in relNonRelDict:
                    relNonRelDict[token] = {"R":0, "N":0}
                relNonRelDict[token][RN] += 1
                    
    return bow, relNonRelDict
bow, relNonRelDict = bowAndRel(paths)

rnrDict = {}
counter = 0
for token, val in relNonRelDict.items():
    if val["N"] == 0:
        counter += 1
        rnrDict[token] = 586 # 이 값을 뭘로 할지가 중요----------------------------------------------------------------------
    else:
        rnrDict[token] = val["R"] / val["N"] 
        # 1. non relative에서 한번도 안나온 단어일땐..? (절반정도)
            # 중요한 단어일 수도있고.. 오타일수도 있고 ㅡㅡ 음..
            # 1. 최댓값으로 설정? (586)
        # 2. 값의 범위가 너무 넓음 ㅡㅡ : 0 ~ 586.. 
            # 로그를 씌워?.. (분모가 더 크면 음수가 되고.. 진수가 1일땐? .. 1.1로?..)

### bow 출력!

In [19]:
displayTupList(sortDict(bow), -1, withVal = True)

size  :  28495   dress  :  20503   wash  :  17105   polyester  :  15792   sleeve  :  13971   imported  :  13475   closure  :  12592   please  :  12319   fit  :  11821   casual  :  11529   cotton  :  11482   spandex  :  11355   wear  :  11336   long  :  11063   machine  :  10291   soft  :  10158   fabric  :  9847   style  :  9755   material  :  9741   color  :  9732   party  :  9634   waist  :  9610   length  :  9558   s  :  9242   dry  :  9166   hand  :  9127   u  :  8693   occasion  :  8392   skirt  :  7538   comfortable  :  7498   inch  :  7316   woman  :  7256   perfect  :  7018   sweater  :  6930   chart  :  6901   design  :  6892   pocket  :  6773   neck  :  6763   zipper  :  6580   front  :  6371   feature  :  6134   high  :  6050   made  :  5973   cold  :  5905   can  :  5815   back  :  5715   look  :  5549   product  :  5423   make  :  5288   top  :  5193   rayon  :  5160   hem  :  4858   quality  :  4722   cardigan  :  4636   daily  :  4516   pull  :  4481   pant  :  4480   gr

  teen  :  357   thigh  :  354   irregular  :  352   put  :  352   half  :  351   petite  :  351   boho  :  349   functional  :  349   leisure  :  346   needed  :  346   yarn  :  345   unlined  :  344   vary  :  344   navy  :  343   ruched  :  343   bra  :  342   green  :  341   option  :  341   self  :  341   using  :  341   beaded  :  340   vest  :  340   xx-large  :  338   excellent  :  337   label  :  337   purchasing  :  337   accessory  :  336   delicate  :  336   various  :  336   slimming  :  335   max  :  334   satin  :  334   range  :  331   set  :  331   however  :  330   included  :  329   manual  :  329   jumper  :  328   embroidered  :  327   nightclub  :  326   fine  :  322   softer  :  321   batwing  :  320   money  :  320   staple  :  320   kimono  :  319   woven  :  318   guaranteed  :  317   experience  :  315   ultra  :  315   festival  :  314   stand  :  314   cross  :  312   friendly  :  312   kind  :  312   loop  :  312   terry  :  312   ironing  :  311   approxi

110   oversize  :  110   square  :  110   stretched  :  110   authenticity  :  109   close  :  109   ireland  :  109   leopard  :  109   o  :  109   romantic  :  109   selling  :  109   today  :  109   classical  :  108   five  :  108   generic  :  108   mom  :  108   produced  :  108   silver  :  108   whole  :  108   completely  :  107   couple  :  107   covered  :  107   degree  :  107   see-through  :  107   dedicated  :  106   ensemble  :  106   flirty  :  106   lace-up  :  106   lift  :  106   ribbon  :  106   error  :  105   mild  :  105   naturally  :  105   say  :  105   appeal  :  104   highly  :  104   organic  :  104   personality  :  104   yoke  :  104   african  :  103   breezy  :  103   indoor  :  103   long-sleeve  :  103   themed  :  103   all-day  :  102   cutout  :  102   fade  :  102   graceful  :  102   outer  :  102   slender  :  102   accent  :  101   aware  :  101   lover  :  101   pc  :  101   proper  :  101   received  :  101   tuck  :  101   wring  :  101   c

 :  46   af  :  46   allowing  :  46   assorted  :  46   bling  :  46   combined  :  46   compared  :  46   cover-up  :  46   crinoline  :  46   exist  :  46   fit-and-flare  :  46   floria  :  46   global  :  46   improved  :  46   introduction  :  46   known  :  46   local  :  46   packed  :  46   partial  :  46   resilient  :  46   several  :  46   sewing  :  46   sleep  :  46   streamlined  :  46   sturdy  :  46   tier  :  46   two-piece  :  46   able  :  45   accentuates  :  45   acetate  :  45   advanced  :  45   beyond  :  45   camo  :  45   cotton-rich  :  45   coverup  :  45   m-bust  :  45   pictured  :  45   rhythm  :  45   s-xxl  :  45   wax  :  45   worth  :  45   www  :  45   college  :  44   conservative  :  44   convertible  :  44   deal  :  44   dog  :  44   doubt  :  44   draping  :  44   dream  :  44   eco-friendly  :  44   engineered  :  44   handle  :  44   ho  :  44   inseams  :  44   johnny  :  44   l-bust  :  44   lock  :  44   melange  :  44   msrp  :  44   nee

bum  :  24   cashmerlon  :  24   checking  :  24   chellysun  :  24   cleavage  :  24   climate  :  24   compact  :  24   competitive  :  24   con  :  24   deserve  :  24   dusty  :  24   dwr  :  24   e  :  24   elbow-length  :  24   elephant  :  24   embrace  :  24   f  :  24   fluid  :  24   foldable  :  24   goat  :  24   h  :  24   harbor  :  24   horizontal  :  24   infringe  :  24   insulator  :  24   inverted  :  24   knowing  :  24   least  :  24   lenght  :  24   measuring  :  24   mocha  :  24   napa  :  24   no-gap  :  24   non-stretch  :  24   on-seam  :  24   origin  :  24   pinup  :  24   pony  :  24   processing  :  24   rabbit  :  24   required  :  24   ruana  :  24   rubbing  :  24   sag  :  24   scale  :  24   showcase  :  24   side-seam  :  24   sincerely  :  24   sleepwear  :  24   tan  :  24   though  :  24   throwing  :  24   toss  :  24   track  :  24   trapeze  :  24   typical  :  24   unauthorized  :  24   uneven  :  24   wearer  :  24   windproof  :  24   work

...

1   ivanka  :  1   ivory-colored  :  1   ivorypink  :  1   izacu  :  1   j-pockets  :  1   j-rose  :  1   j-sleeve  :  1   jacquard-patterned  :  1   jacquard-texture  :  1   jacquard-weave  :  1   jacquare  :  1   jag  :  1   jagged-edge  :  1   jam  :  1   james  :  1   jamie  :  1   jammies  :  1   jamming  :  1   jan  :  1   jancure  :  1   jane  :  1   janes  :  1   janeswith  :  1   janie  :  1   japanese-style  :  1   jaquard  :  1   jasmine  :  1   jayjay  :  1   jayla  :  1   jc  :  1   jczhwqu  :  1   jdj  :  1   jeagging  :  1   jean-ius  :  1   jelly  :  1   jersey-lined  :  1   jersey-stitched  :  1   jescakoo  :  1   jess  :  1   jet-set  :  1   jevenis  :  1   jewelry-clasp  :  1   jewnicorn  :  1   jk  :  1   jockey  :  1   jodifl  :  1   joe  :  1   joined  :  1   jointed  :  1   jolliest  :  1   jomtong  :  1   jordyn  :  1   jorlye  :  1   joyfully  :  1   joymode  :  1   jq  :  1   jughead  :  1   jumpauit  :  1   jumpuist  :  1   junarose  :  1   june  :  1   junio

 1   midiskirt  :  1   midlength  :  1   midsole  :  1   midthigh  :  1   midway  :  1   mightwear  :  1   mikado  :  1   mila  :  1   milan  :  1   milder  :  1   mildew  :  1   mille  :  1   milly  :  1   milt-occasions  :  1   milu  :  1   mimetic  :  1   mimic  :  1   mimic-silk  :  1   mimicking  :  1   mimosa  :  1   minding  :  1   mindset  :  1   mine  :  1   mingle  :  1   mingxintech  :  1   mini-rib  :  1   minibee  :  1   minimalistic  :  1   minimally  :  1   minion  :  1   mintleaf  :  1   minuts  :  1   mira  :  1   miranda  :  1   misc  :  1   misleading  :  1   mismatched  :  1   miter  :  1   mitered  :  1   mitring  :  1   mix-stitch  :  1   mixed-eyelet  :  1   mixed-stitch  :  1   mixed-yarn  :  1   mixi  :  1   mizoci  :  1   mk  :  1   mmt  :  1   mo  :  1   moda  :  1   modacrylic  :  1   model-height  :  1   moden  :  1   modernizes  :  1   modestry  :  1   modification  :  1   modifying  :  1   mody  :  1   moisture-absorbing  :  1   moisture-releasing  :  1  

polyester-cotton-lycra  :  1   polyester-spandex  :  1   polyester-type  :  1   polyesterlight  :  1   polyestr  :  1   polyestter  :  1   polyethylene  :  1   polyetser  :  1   polymiade  :  1   polymide  :  1   polytaffeta  :  1   polyurathane  :  1   polyurethane-coated  :  1   polyvinyl  :  1   pomegranate  :  1   pompom-tipped  :  1   pompon  :  1   poncho-like  :  1   ponti  :  1   poofy-looking  :  1   poor-quality  :  1   poorboy  :  1   poorly  :  1   pop-it-on-over-anything  :  1   pop-on  :  1   popart  :  1   popbeauty  :  1   porcini  :  1   porckets  :  1   porosity  :  1   portfolio  :  1   portrays  :  1   portugal  :  1   porvide  :  1   positioned  :  1   possessing  :  1   post-baby  :  1   post-cure  :  1   post-second  :  1   postage  :  1   poster  :  1   posterior  :  1   postpartum  :  1   potruding  :  1   pouring  :  1   pours  :  1   power-shaping  :  1   poy  :  1   poylester  :  1   poylster  :  1   ppockets  :  1   pr  :  1   practitioner  :  1   prairie  

  :  1   sky-scraping  :  1   skye  :  1   skynny  :  1   skyrider  :  1   slack-style  :  1   slacksor  :  1   slanting  :  1   slask  :  1   slay  :  1   slde  :  1   sled  :  1   sleeker  :  1   sleeless  :  1   sleep-dress  :  1   sleepshirt  :  1   sleev  :  1   sleeve-black  :  1   sleevee  :  1   sleeveles  :  1   sleevelss  :  1   sleevepleated  :  1   sleever  :  1   sleevet-shirt  :  1   sleevewear  :  1   sleevs  :  1   sleeze  :  1   slendering  :  1   slenderize  :  1   sleuth  :  1   slevee  :  1   slevees  :  1   sleveless  :  1   slide-in  :  1   slightest  :  1   slightlt  :  1   slightly-fluttering  :  1   slihouette  :  1   slim-figure  :  1   slim-fitted  :  1   slim-regular  :  1   slim-skinny  :  1   slim-yet-easy  :  1   slimiar  :  1   slimmer-fitting  :  1   slimmer-looking  :  1   slimness  :  1   slinnky  :  1   slipped  :  1   slipt  :  1   slite  :  1   slited  :  1   sliver  :  1   slob  :  1   slock  :  1   slong  :  1   slothful  :  1   slowing  :  1   s

technician  :  1   technics  :  1   technolody  :  1   tecno  :  1   teddyfleece  :  1   tediousness  :  1   tee-shirt-like  :  1   teenage  :  1   teh  :  1   tel  :  1   telas  :  1   telling  :  1   temper  :  1   temperate  :  1   temperater  :  1   temperatur  :  1   template  :  1   temple-ready  :  1   tempting  :  1   temptress  :  1   tencel-lyocell  :  1   tencole  :  1   tendency  :  1   tenderly  :  1   tener  :  1   tense-knitted  :  1   tent  :  1   tenth  :  1   tention  :  1   tequila  :  1   termal  :  1   terrain  :  1   terrence  :  1   terrific  :  1   terylence  :  1   tessel  :  1   testament  :  1   tested  :  1   texeresilk  :  1   textture  :  1   texture-knit  :  1   textured-knit  :  1   texturize  :  1   thackery  :  1   thame  :  1   thanks-giving  :  1   thanksgving  :  1   thatfs  :  1   thats  :  1   the-off-shoulder  :  1   thelovely  :  1   thematic  :  1   theory  :  1   thereafter  :  1   therir  :  1   therma-fit  :  1   thermally  :  1   thermatech

In [22]:
print(counter, " / ", len(rnrDict))  # 딱 절반의 단어가 non-rel 에는 등장하지 않음.. 
# 8209 / 17509
# 8205 / 17388  (stopword 제거)
# 7571 / 15954 (lemmatize - 표제어 추출)

7571  /  15954


In [9]:
json.dump(bow, open(f'/home/jovyan/work/분석/bow' + '.json', 'w+')) 
json.dump(rnrDict, open(f'/home/jovyan/work/분석/relNonRel' + '.json', 'w+')) 

### r/nr 출력!

In [8]:
displayTupList(sortDict(rnrDict), -1, withVal = True)

# 1. idf가 낮을것 (여기저기 잘 등장할것) + r/n 값이 클것 (상의에만 잘 등장할 단어일것)
# 2. 고민) NonRel = 0 인경우.. rel에만 등장한 단어인데.. (거의 절반임) 
# =======>  주로, 의미없는단어, 오타 등등인 경우가 많음..

a-grade  :  586   a-lien  :  586   a-little-long  :  586   a-z  :  586   aamzon  :  586   ababalaya  :  586   abaya  :  586   abberation  :  586   abbreviation  :  586   abd  :  586   abes  :  586   ablaze  :  586   abluent  :  586   abollria  :  586   above-knees  :  586   above-the-ankle  :  586   absoluely  :  586   absorbant  :  586   absorbency  :  586   absord  :  586   absortion  :  586   abundant  :  586   abvoe  :  586   abyssian  :  586   ac  :  586   academic  :  586   accasion  :  586   accassion  :  586   accation  :  586   accations  :  586   accebtuate  :  586   acceessories  :  586   accentuating  :  586   acceptable  :  586   accepted  :  586   access-pull  :  586   accessibility  :  586   accessorized  :  586   accessorizes  :  586   accessorizing  :  586   accident  :  586   accodring  :  586   accoridng  :  586   accpet  :  586   accurately  :  586   accylic  :  586   acharline  :  586   ackets  :  586   acomfortable  :  586   acryli  :  586   acrylic-fluffy  :  586

   bateau  :  586   bateau-neck  :  586   bath  :  586   bathing-suit  :  586   bathrobe  :  586   bating  :  586   batswing  :  586   batwing  :  586   bavaria  :  586   bawing  :  586   bawting  :  586   bay  :  586   bbalizko  :  586   bbqs  :  586   bc  :  586   bca  :  586   bcakless  :  586   be  :  586   beaach  :  586   beacause  :  586   beachcoco  :  586   beachdress  :  586   bead-embellished  :  586   bead-work  :  586   beadwork  :  586   bean  :  586   beard  :  586   bearing  :  586   beast  :  586   beat  :  586   beaten  :  586   beatifully  :  586   beause  :  586   beaut  :  586   beautidul  :  586   beautife  :  586   beautifull  :  586   beautifully-fitting  :  586   beautifully-made  :  586   beautify  :  586   beautigul  :  586   bebonnie  :  586   becase  :  586   becuase  :  586   bedazzled  :  586   bedminton  :  586   bedroom  :  586   bedtime  :  586   bedwear  :  586   bee  :  586   beechwood  :  586   beer  :  586   beet  :  586   befitting  :  586   befor

casvading  :  586   catagory  :  586   category  :  586   cater  :  586   caters  :  586   cathering  :  586   catsuit  :  586   caual  :  586   caught  :  586   causing  :  586   causul  :  586   cautious  :  586   caviar  :  586   cavity  :  586   cayyon  :  586   cb  :  586   ccasion  :  586   ccasual  :  586   ccfw  :  586   cd  :  586   cdeep  :  586   ceates  :  586   cedar  :  586   celcius  :  586   celeb  :  586   celebation  :  586   celina  :  586   celitas  :  586   cellulose  :  586   celtic  :  586   centerback  :  586   centered  :  586   centerstage  :  586   century  :  586   certainly  :  586   certificate  :  586   certifying  :  586   cetner  :  586   cfm  :  586   cha  :  586   chafe-free  :  586   challa  :  586   chaming  :  586   champage  :  586   chandelier  :  586   changeable  :  586   changeful  :  586   changuan  :  586   chapter  :  586   characteristically  :  586   characterized  :  586   charistmas  :  586   charitable  :  586   charleston  :  586   ch

   crazy  :  586   crb  :  586   crease-resistant  :  586   creat  :  586   createing  :  586   creator  :  586   creats  :  586   creeping  :  586   crest  :  586   crest-embossed  :  586   crew-neck  :  586   crew-neckline  :  586   creweneck  :  586   crewnec  :  586   crewneck  :  586   crewneeck  :  586   crinkle-dot  :  586   cris  :  586   criss-crossed  :  586   crisscrossing  :  586   crissscross  :  586   cro  :  586   crochet-look  :  586   crochet-trim  :  586   crocheting  :  586   crocodelle  :  586   cronin  :  586   crop-sleeve  :  586   cropprd  :  586   cross-back  :  586   cross-line  :  586   cross-shirring  :  586   cross-stitch  :  586   cross-v-neck  :  586   cross-wrap  :  586   crossbones  :  586   crossed  :  586   crossv  :  586   crosswind  :  586   crow  :  586   crowded  :  586   crowlneck  :  586   crowned  :  586   crpped  :  586   crt  :  586   cruelty  :  586   cruising  :  586   crunchy  :  586   crus-lenth  :  586   crushing  :  586   crystalline  : 

duran  :  586   dure  :  586   dust  :  586   duster-style  :  586   dustyplum  :  586   dustysage  :  586   dwon  :  586   dydsz  :  586   dye-sublimation  :  586   dye-to-match  :  586   dying  :  586   dylan  :  586   dymaisei  :  586   dynen  :  586   dz  :  586   e-mail  :  586   e-packet  :  586   eachever  :  586   eagle  :  586   earily  :  586   earned  :  586   earning  :  586   earnings  :  586   earphone  :  586   eart  :  586   easiry  :  586   east  :  586   easther  :  586   eastic  :  586   easy-chic  :  586   easy-fit  :  586   easy-match  :  586   easy-moving  :  586   easy-off  :  586   easy-on  :  586   easy-peasy  :  586   easy-to-match  :  586   easy-wearing  :  586   eating  :  586   eature  :  586   eatures  :  586   eblow  :  586   eccentric  :  586   eccentricity  :  586   ecept  :  586   echo  :  586   eclectic  :  586   eco-fleece  :  586   eco-jersey  :  586   eco-space  :  586   ecolivzit  :  586   ecological  :  586   economic  :  586   economically  :  5

...

  :  1.5714285714285714   trusted  :  1.5714285714285714   max  :  1.5692307692307692   fair  :  1.5647058823529412   choies  :  1.56   smooth  :  1.5588235294117647   clasp  :  1.5555555555555556   packed  :  1.5555555555555556   peasant  :  1.5555555555555556   all-day  :  1.55   tulip  :  1.55   fading  :  1.5454545454545454   grid  :  1.5454545454545454   thereby  :  1.5454545454545454   pump  :  1.5416666666666667   drawstring  :  1.536624203821656   case  :  1.5357142857142858   accentuate  :  1.5294117647058822   optional  :  1.5294117647058822   classic  :  1.5281124497991967   anything  :  1.5263157894736843   hip  :  1.5208775654635527   loved  :  1.5161290322580645   tiered  :  1.510204081632653   poly  :  1.5054945054945055   account  :  1.5   adjusted  :  1.5   amp  :  1.5   appreciated  :  1.5   banding  :  1.5   beckham  :  1.5   behind  :  1.5   bias  :  1.5   bringing  :  1.5   buttom  :  1.5   chafe  :  1.5   checkerboard  :  1.5   chilli  :  1.5   cinching  :  1.5   

 1.0555555555555556   n  :  1.0555555555555556   detergent  :  1.0526315789473684   quantity  :  1.0526315789473684   woven  :  1.0516129032258064   wax  :  1.0454545454545454   www  :  1.0454545454545454   buckle  :  1.0434782608695652   airy  :  1.032258064516129   blazer  :  1.0289855072463767   festival  :  1.0258064516129033   fastening  :  1.0238095238095237   faux  :  1.0220588235294117   play  :  1.0212765957446808   four  :  1.0158730158730158   panel  :  1.0097560975609756   a-lina  :  1.0   a-lined  :  1.0   aanything  :  1.0   abrasion  :  1.0   academy  :  1.0   accessible  :  1.0   accroding  :  1.0   act  :  1.0   adjustbale  :  1.0   adjusts  :  1.0   afibi  :  1.0   agan  :  1.0   air-dry  :  1.0   alexa  :  1.0   all-cotton  :  1.0   alongside  :  1.0   amber  :  1.0   ample  :  1.0   anchor  :  1.0   annflat  :  1.0   anymore  :  1.0   apparent  :  1.0   apperal  :  1.0   appropriately  :  1.0   appx  :  1.0   armor  :  1.0   aroud  :  1.0   arrives  :  1.0   artisan

ribbon  :  0.9272727272727272   khaki  :  0.9264705882352942   camel  :  0.9230769230769231   direction  :  0.9230769230769231   steal  :  0.9230769230769231   valuable  :  0.9230769230769231   move  :  0.9186046511627907   bump  :  0.9166666666666666   lead  :  0.9166666666666666   multi-color  :  0.9166666666666666   nightout  :  0.9130434782608695   sometimes  :  0.9090909090909091   spanx  :  0.9090909090909091   defect  :  0.9   golden  :  0.9   lenth  :  0.9   tuxedo  :  0.9   component  :  0.8947368421052632   fitness  :  0.8947368421052632   importado  :  0.8947368421052632   opaque  :  0.8947368421052632   concern  :  0.8928571428571429   slip-on  :  0.8928571428571429   glamorous  :  0.8913043478260869   extender  :  0.8888888888888888   led  :  0.8888888888888888   perspective  :  0.8888888888888888   safety  :  0.8888888888888888   movement  :  0.8863636363636364   lighter  :  0.8823529411764706   slim-fit  :  0.8823529411764706   application  :  0.875   breeze  :  0.875   

0.5   high-waisted  :  0.5   hlbandage  :  0.5   iheartraves  :  0.5   imysty  :  0.5   inc  :  0.5   inelastic  :  0.5   infinite  :  0.5   infor  :  0.5   jazz  :  0.5   jessica  :  0.5   jet  :  0.5   kelly  :  0.5   laser-cut  :  0.5   lengh  :  0.5   light-weighted  :  0.5   mandala  :  0.5   materil  :  0.5   menswear  :  0.5   mock-wrap  :  0.5   multi-pocket  :  0.5   navel  :  0.5   no-wrinkle  :  0.5   non-itch  :  0.5   non-see-through  :  0.5   odering  :  0.5   omni-shade  :  0.5   opportunity  :  0.5   orientation  :  0.5   paint  :  0.5   panelled  :  0.5   partly  :  0.5   patriotism  :  0.5   pear  :  0.5   penchant  :  0.5   persun  :  0.5   pieced  :  0.5   pillow  :  0.5   pinstriped  :  0.5   playsuit  :  0.5   pleae  :  0.5   portrait  :  0.5   precise  :  0.5   processed  :  0.5   prolonged  :  0.5   pulp  :  0.5   radiation  :  0.5   rag  :  0.5   raw-cut  :  0.5   republic  :  0.5   reserve  :  0.5   restraint  :  0.5   retro-styled  :  0.5   returned  :  0.5  

0.18181818181818182   imperfection  :  0.18181818181818182   luscious  :  0.18181818181818182   minimize  :  0.18181818181818182   pah  :  0.18181818181818182   pjs  :  0.18181818181818182   raan  :  0.18181818181818182   dp  :  0.17647058823529413   slims  :  0.17543859649122806   esteez  :  0.1693548387096774   slant  :  0.16822429906542055   accordion  :  0.16666666666666666   alter  :  0.16666666666666666   artistic  :  0.16666666666666666   bodysuits  :  0.16666666666666666   chaus  :  0.16666666666666666   cigarette  :  0.16666666666666666   d-ring  :  0.16666666666666666   double-stiched  :  0.16666666666666666   elasterell-p  :  0.16666666666666666   embarrassment  :  0.16666666666666666   favor  :  0.16666666666666666   high-stretch  :  0.16666666666666666   hip-hop  :  0.16666666666666666   iphones  :  0.16666666666666666   klimt  :  0.16666666666666666   lengthening  :  0.16666666666666666   leotard  :  0.16666666666666666   put-together  :  0.16666666666666666   sagging  : 

## 02. tf idf 계산

In [11]:
#path 에 있는 모든 id.json 파일을 가지고, tf를 구하여 "ID" : tf   꼴의 딕셔너리 리턴
#path 여러개로 재정의
def computeAllTF(paths):
    tfDict = {} 
    for path in paths:
        for item in path.iterdir():
            if(item.is_dir()): #안보이는 디렉토리가 있다.. 주의 ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
                continue
            itemJson = json.loads(item.read_text(), encoding='utf-8')
            features = itemJson["features"]
            ID = itemJson["id"]    
            tf = computeTF(bow, features)
            tfDict[ID] = tf
    return tfDict

# ---------------메인1(tf 전부 계산)---------------
tfDict = {} #tf 들을 저장
tfDict = computeAllTF(paths)

# ______메인2(idf 계산)___________
#idf = computeIDF(tfDict)

In [26]:
# 저장! (또 계산하지말고 불러쓰자)
json.dump(idf, open(f'/home/jovyan/work/분석/idf' + '.json', 'w+')) 
print(len(idf)) #17388 #15954
json.dump(tfDict, open(f'/home/jovyan/work/분석/TFs' + '.json', 'w+')) 
print(len(tfDict)) #31318 

15954
31318


### idf 출력!

In [21]:
displayTupList(sortDict(idf), -1)

a-grade    a-ling    a-little-long    aamzon    ababalaya    abberation    abes    ablaze    above-knees    above-the-ankle    abrasion-resistant    absence    absoluely    absorbant    absorbs    absord    absortion    abundant    abyssian    academic    accassion    accation    accations    accebtuate    accesorios    accesorized    accesorizing    access-pull    accessibility    accessorizes    accodring    accompanies    accounting    accylic    acessories    achieved    ackets    acomfortable    acre    acryl    acrylic-fluffy    acrylic-made    acrylique    acrylon    acrytic    acryylic    acticities    activedry    activies    activitiesetc    actu    acturally    ad    adaptar    adcanced    addict    addjust    addressee    adele    adjacent    adjuested    adjustble    adjustement    adjuster    adjutable    admiring    adn    ado    adopting    adorning    adriano    adrienne    adventuring    adversity    aerobics    aesthetically    afer    affecting    aficianado    afri

   cahrt    caidgan    caifeixiang    cailami    calculate    caldore    caled    calendar    calf-ankle    calf-hugging    cali    caliadress    calibrated    calling    calming    calorie    cames    camisloes    camisol    camoflage    camois    canalside    cancel    candigan    candlelight    cane-inspired    cangaroo    cannabis    cansual    cantact    cape-back    cape-style    capital    captain    captivate    captured    carabiner    caradigan    caravan    carbon-black    carbon-fibre    cardigan-stitch    cardiganis    cardigant    cardigian    cardigon    cardin    cardinal    care-free    careen    career-chic    carelabel    carftsmanship    cargigan    cariety    carolina    caroling    carpus    carrera    carrot-leg    cartigan    cartigans    carwash    cashmere-cotton    cassical    casuail    casual-cool    casual-cute    casualbeachpartyworkoustsideusually    casuales    casuall    casualy    casulal    categorie    cathering    catsuit    causing    cautious    

    euramerican    eurivicy    euro-market    eva    evaluation    evelust    eveningparty    eveningwear    evenly    everbody    everning    eververy    evev    evevated    evneing    evo    evolve    ex    exceeding    excelente    excellente    excels    excessively    exchage    excite    exit    exiting    exo-web    expecte    expediated    expedite    expeited    expense    experice    experiencelover    explains    explicit    explode    explore    export    expressing    exquisiteness    exsit    extende    extender-polyester    extending    extermely    externally    extra-fine    extra-high    extra-quickly    extra-strong    extrafine    extraordinarily    extravagance-this    extremly    eye-catcher    eye-cathing    eye-getting    eye-let    eye-popping    eyeball    eyelash-fringe    eyelash-trim    eyelet-trimmed    ez    fabolous    fabri    fabric-on    fabrica    fabricado    fabricwe    fabris    fabulosa    facture    faddare    fadded    fade-free    fade-resista

   lemontrip    length-same    length-us    leo    leoflage    leonard    leopard-printed    leopardd    lepard    leporad    lesotho    lesson    lest    letsrunwild    lettuce    leviathan    lewego    lf    lfet    lfpo    lghtweight    liangsm    liberate    liberte    lick    lides    lifeguard    lifelike    lifelong    lifeshe    lift-hips    ligament    ligh    light-wash    lightning    lightwear    lightwegith    lightweightt    lightweigt    lightweigtht    lightwight    lignt    ligthtweight    liitle    like-a-glove    liked    likewise    likre    lil    liling    limited-time    limming    limted    limtery    linda    lined-for    lineing    linen-blend    linen-like    lingering    liningand    liningprincess    linining    linling    linsis    lint    lint-free    lipps    liquid-looking    liquidation    literary    lithesome    livin    liying    liz    lladies    llamitas    lleva    lliy    lmake    lna    locator    locomolife    logan    logic    logo-embroidere

    openning    opent    openweave    opp    opportunely    oppose    opted    optimizing    option-it    optionsavailable    optton    order-to-made    orderening    ordermeasurement    ordry    organdy    organization    organizer    oriental    origami    orita    ormay    ornamentation    oro    orvia    orwinter    oscillate    otherapparel    ou    ouija    oukers    ouncuty    ound    ourfit    ourstanding    out-going    out-of-date    out-of-style    out-of-trend    outccol    outcome    outcool    outdate    outdo    outear    outer-layer    outerwar    outfashion    outfit-displaying    outfitl    outgrown    outlaw    outlay    outlet    outlining    outmode    outrageous    outseams    outsole    outstiched    outwdoor    over-up    overall-style    overheated    overjoyed    overlaped    overlaping    overlapped    overlayer    overlaying    overloaded    overlock    overlocked    overlocking    overly-revealing    overpowering    oversid    overside    overskirt    overs

...

   absorbing    accasions    accuracy    adapt    afraid    all-around    all-purpose    angora    appealing    apple    appreciate    aprox    array    back-zip    backing    beluring    bodyline    bonded    boost    boulder    brandnameinternal    bronze    came    carraig    charity    chi    coloured    comfortale    comfotable    concise    conditioning    couch    cousin    coz    culture    current    cut-outs    deep-v    demin    demure    developed    device    donated    donn    double-layer    drone    easy-wear    eliacher    entirely    enusre    era    ermonn    eventually    everybody    everytime    fabic    falcon    fawn    fight    fishnet    floaty    flouncy    footwear    frabic    fyi    germany    get-together    gurantee    hail    headband    hiden    hipline    ho    honest    hunting    hurry    importantly    impress    interlining    interview    ironed    joycheer    july    l-xl    laptop    laundered    license    ligthweight    lilac    maintaining  

bone    breathe    brighter    cartoon    champagne    characterized    clavicle    complaint    conformtable    conned    covering    cross-over    crushed    cyber    darker    defect    density    destination    dicky    dress-up    drink    dusty    elevate    evenning    faded    faster    faux-wrap    flawless    forget    greek    ground    hanukkah    highest-quality    holding    honeycomb    human    indoors    isle    knock-off    ladylike    lalagen    lifetime    lightly    lilly    marathon    marycrafts    mentioned    minimalist    negative    often    outseam    passionate    peek    pile    pinstripe    pom    pre-shrinked    proportion    pulitzer    relationship    renaissance    reproduces    rinse    risk-free    roll-up    sacrificing    sakkas    sharp    shimmer    six    skim    slide    snuggly    spotlight    storing    stroll    sumptuous    taper    tartan    timely    top-stitched    tourism    traffic    unicorn    upto    volume    weeding    wider    a

displayed    engineered    ethically    factor    happiness    internal    kick    mint    mohair    opaque    opposite    pan    piping    protects    puffy    remember    retain    rub    scuba    snow    staff    stock    stocking    stone    tweed    urban    wait    wax    afternoon    arrived    beyond    bound    brightness    butt    coin    done    easy-going    flattens    forgiving    instant    lamb    longest    longevity    morning    object    plenty    post    purse    raanpahmuang    reasonable    recovery    reshape    rivet    romper    sculpts    smoothly    stretching    synthetic    tighter    upgrade    advice    appear    asin    bachelorette    camo    chill    component    contouring    counterfeiter    experienced    hiding    importado    looser    mercantile    nude    ocassion    plate    purchased    reach    responsible    secretly    slanted    structure    taped    anthropologie    antique    burnout    cascade    chenille    chino    concerned    deca

   offered    reseller    square    breezy    craftmanship    girlfriend    lover    matte    mid-rise    occassion    seperately    softly    yoke    bold    ethnic    jogger    long-sleeve    pc    star    tencel    underneath    width    accent    aware    degree    expedited    flirty    oversize    received    slender    stretched    therefore    all-day    graceful    heart    indoor    petticoat    ribbon    romantic    utmost    wring    covered    error    fade    groom    personality    table    flow    lace-up    say    see-through    appeal    classical    completely    five    highly    jewelry    roll    tights    today    artisan    ensemble    mild    naturally    ceremony    dedicated    outerwear    silver    close    fluffy    o    update    whole    brings    email    generic    produced    selling    sense    drying    effortlessly    non    polyurethane    room    charcoal    garden    heather    interior    serial    tab    thickness    all-match    bridal    col

regular    new    washing    contact    vintage    belt    dating    collar    cozy    well    relaxed    just    blend    may    basic    wide    plus    featuring    heel    full    tunic    crew    height    simple    shopping    guarantee    silhouette    a-line    large    elastane    zip    lining    light    unique    chic    carefully    iron    maxi    bottom    also    drawstring    beautiful    best    left    shirt    black    ribbed    flat    like    recommended    image    round    medium    pattern    available    keep    leg    cute    boot    sleeveless    flattering    prom    v    school    see    due    sweatshirt    special    home    different    date    floral    season    two    ordering    comfort    brand    night    lace    cuff    suit    show    formal    note    usa    good    waistband    holiday    match    item    comfy    autumn    etc    t    stylish    vacation    feel    viscose    body    super    picture    breathable    slim    line    neckline 

# * 불러오기

In [2]:
bow = json.loads(Path('/home/jovyan/work/분석/bow.json').read_text(), encoding='utf-8')
rnrDict = json.loads(Path('/home/jovyan/work/분석/relNonRel.json').read_text(), encoding='utf-8')
tfDict = json.loads(Path('/home/jovyan/work/분석/TFs.json').read_text(), encoding='utf-8')
idf = json.loads(Path('/home/jovyan/work/분석/idf.json').read_text(), encoding='utf-8')

In [ ]:
displayTupList(sortDict(idf), -1)#값의 범위 : 0 ~ 4

# 03. tf * idf * R/NR 

In [11]:
def computeTFIDF_R(tfBow, idfs,rnr): 
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val * idfs[word] * rnr[word]
    return tfidf
def computeTFIDF2_R(tfBow, idfs,rnr):
    idftf = {} 
    for word, val in tfBow.items():
        if idfs[word] == 0:
            idftf[word] = 0
        else:
            idftf[word] = val / idfs[word] * rnr[word] 
    return idftf
tfidfDict = {} #  {ID : tf- * idf}
tfidfDict2 = {} # {ID : idf / tf}
tfidfDict_R = {} # {ID : tf * idf * rNr}
tfidfDict2_R = {} # {ID : idf / tf * rNr}

counter = 0
for item in path1.iterdir():
    if(item.is_dir()): #안보이는 디렉토리 주의
        continue
    itemJson = json.loads(item.read_text(), encoding='utf-8')
    features = itemJson["features"]
    ID = itemJson["id"]
    #-------------------------(일반 : tf * idf)-----------------------
    tfidf = computeTFIDF(tfDict[ID], idf)
    tfidfDict[ID] = tfidf
    #-------------------------(시도1 : tf / idf)--------------------
    tfidf2 = computeTFIDF2(tfDict[ID], idf)
    tfidfDict2[ID] = tfidf2
    #------------------------(시도2 : tf * idf * RNR)------
    tfidf_R = computeTFIDF_R(tfDict[ID], idf, rnrDict)
    tfidfDict_R[ID] = tfidf_R
    #-----------------------(시도3: tf / idf * RNR)-------
    tfidf2_R = computeTFIDF2_R(tfDict[ID], idf, rnrDict)
    tfidfDict2_R[ID] = tfidf2_R
    #-----------------------(출력)-----------------------------------------
    print("ID = " , ID, "======================")
    print("<features>")
    print(features)
    #print("< TF >")
    #displayTupList(sortDict(tfDict[ID]), -1)
    print(" < TF x IDF > ")
    displayTupList(sortDict(tfidf), 5)
    print(" < TF X IDF X RNR >")
    displayTupList(sortDict(tfidf_R), 5)
    print(" < TF / IDF >")
    displayTupList(sortDict(tfidf2), 5)
    print(" < TF / IDF * RNR >")
    displayTupList(sortDict(tfidf2_R), 5)
    counter += 1
    if counter == 30:
        break

ID =  B009S3IAU2 ======================
<features>
['100% Polyester', 'Imported', 'Dry Clean Only', 'Empire-waist gown featuring ruched bust and concealed side zipper']
 < TF x IDF > 
empire-waist    ruched    gown    concealed    featuring    
 < TF X IDF X RNR >
empire-waist    bust    gown    ruched    concealed    
 < TF / IDF >
polyester    imported    dry    zipper    clean    
 < TF / IDF * RNR >
empire-waist    bust    polyester    gown    imported    
ID =  0223688975 ======================
<features>
['Material :100%Polyester', 'Feature:Long Sleeve,1/4 Zipper,Long Length,Hooded,Pocket, Color Block /Sherpa/Fleece/Pullover/Sweater/Sweatshirts/Outwear/Jacket/Coat', 'Suitable for daily wear,work,school,dates,sports,vacation,street wear. Pair with Jeans or Legging & Heels/Booties', 'Season: Spring, Autumn and Winter.', 'Wash Recommended With Cold Water/Do Not Bleach/Hang Or Line Dry']
 < TF x IDF > 
booty    sherpa    outwear    block    fleece    
 < TF X IDF X RNR >
sherpa    ho

bandeau    embellished    china    nylon    rayon    
 < TF X IDF X RNR >
neck    sleeve    embellished    long    nylon    
 < TF / IDF >
sleeve    long    neck    rayon    nylon    
 < TF / IDF * RNR >
sleeve    neck    long    embellished    nylon    
ID =  B009MPNJP2 ======================
<features>
['90% Rayon/10% Spandex', 'Made in US', 'Pull On closure', 'Machine Wash', 'Elbow-sleeve dress in A-line silhouette featuring scoop neckline', '36-inch length']
 < TF x IDF > 
elbow-sleeve    scoop    featuring    silhouette    a-line    
 < TF X IDF X RNR >
elbow-sleeve    neckline    scoop    dress    u    
 < TF / IDF >
wash    closure    spandex    machine    dress    
 < TF / IDF * RNR >
neckline    elbow-sleeve    dress    scoop    wash    


### 특정 상품 하나만 출력

In [22]:
def computeAll(ID):
    p = '/home/jovyan/work/크롤링/relFeatures/' + ID + '.json'
    item = Path(p)
    if not item.exists():
        print("미안..그건 없어")
        return False
    itemJson = json.loads(item.read_text(), encoding='utf-8')
    features = itemJson["features"]
    ID = itemJson["id"]
    #-------------------------(일반 : tf * idf)-----------------------
    tfidf = computeTFIDF(tfDict[ID], idf)
    tfidfDict[ID] = tfidf
    #-------------------------(시도1 : tf / idf)--------------------
    tfidf2 = computeTFIDF2(tfDict[ID], idf)
    tfidfDict2[ID] = tfidf2
    #------------------------(시도2 : tf * idf * RNR)------
    tfidf_R = computeTFIDF_R(tfDict[ID], idf, rnrDict)
    tfidfDict_R[ID] = tfidf_R
    #-----------------------(시도3: tf / idf * RNR)-------
    tfidf2_R = computeTFIDF2_R(tfDict[ID], idf, rnrDict)
    tfidfDict2_R[ID] = tfidf2_R
    #-----------------------(출력)-----------------------------------------
    print("ID = " , ID, "======================")
    print("<features>")
    print(features)
    #print("< TF >")
    #displayTupList(sortDict(tfDict[ID]), -1)
    print(" < TF x IDF > ")
    displayTupList(sortDict(tfidf), 5, withVal = True)
    print(" < TF X IDF X RNR >")
    displayTupList(sortDict(tfidf_R), 5, withVal = True)
    print(" < TF / IDF >")
    displayTupList(sortDict(tfidf2), 5, withVal = True)
    print(" < TF / IDF * RNR >")
    displayTupList(sortDict(tfidf2_R), 5, withVal = True)
computeAll("B005NJAG0C")

ID =  B005NJAG0C ======================
<features>
['100% Cotton', 'Imported', '100% Cotton', 'Open front cardigan, Long body length', 'No front closure, Oversized fit', 'Patch pockets', 'Material: cotton rib knit']
 < TF x IDF > 
patch  :  1.966877319497213   rib  :  1.9069622941806605   oversized  :  1.6402748641070675   body  :  1.210236710767094   open  :  1.1812947918017163   
 < TF X IDF X RNR >
cardigan  :  187.40774582122015   oversized  :  108.66820974709321   open  :  43.635212232552625   rib  :  26.105656234128354   knit  :  7.852764815628881   
 < TF / IDF >
imported  :  2.67987128470969   closure  :  2.119905984647802   cotton  :  1.9674899165431645   fit  :  1.8376774848918087   long  :  1.766092241808805   
 < TF / IDF * RNR >
cardigan  :  155.48852759778714   oversized  :  40.38957216848236   open  :  31.26946956408978   long  :  19.894970364766372   knit  :  7.718568177855645   


### category 고치기

In [ ]:
{'정상1': 17911, '정상2': 0, '정상3': 0, '드레스': 8570, '바지': 4066, '새드레스': 434, '머냐': 337}

path = Path('/home/jovyan/work/크롤링/relFeatures')
path2 = Path('/home/jovyan/work/크롤링/nonRelFeatures')
paths = [path, path2]

docs = []
emptyList = []
pathCounter = 0
exCounter = 0
dic = {'정상1' : 0, '정상2':0, '정상3':0, '드레스': 0, '바지' :0, '새드레스':0, '머냐':0}
for path in paths:
    pathCounter += 1 
    for item in path.iterdir():
        if(item.is_dir()): 
            continue
        itemJson = json.loads(item.read_text(), encoding='utf-8')
        try:
            cates = itemJson["fullCategory"]
            ID = itemJson["id"]
        except:
            #B01AA6PPXU #미쳤나..;;이게 왜 섞였지.. 나중에 삭제나 수정 
            continue
        category = itemJson["category"]
        start = ""
        for cate in cates:
            start += cate
        if category == "sweater" or category == "hoodies&sweatshirts" or category == "skirt": 
            #정상적으로 카테고리 지정 됨
            dic["정상1"] += 1
            continue
        elif start.lower().find("pants") != -1:
            itemJson["category"] = "pants"
            json.dump(itemJson, open(f'/home/jovyan/work/크롤링/nonRelFeatures/' + ID + '.json', 'w+')) 
            #바지주인 찾음
            dic["바지"] += 1
        elif start.lower().find("dress") != -1:
            itemJson["category"] = "dress"
            dic["드레스"] += 1
            json.dump(itemJson, open(f'/home/jovyan/work/크롤링/relFeatures/' + ID + '.json', 'w+')) 
            #드레스주인찾음
        else:
            if start.strip() == "":
                emptyList.append(ID)
                #나중에 모아서 한번에 지우자..
            elif pathCounter == 1:
                itemJson["category"] = "dresse"
                json.dump(itemJson, open(f'/home/jovyan/work/크롤링/relFeatures/' + ID + '.json', 'w+'))
                dic["새드레스"] += 1
            elif pathCounter == 2: #출처를 알 수 없는 녀석들은.. nonRel이라고 만들어버림
                itemJson["category"] = "nonRel"
                dic["머냐"] += 1
                print(ID, " , ", start)
                #json.dump(itemJson, open(f'/home/jovyan/work/크롤링/nonRelFeatures/' + ID + '.json', 'w+')) 
print(dic)
